### Data Ingestion

In [1]:
import os

In [2]:
pwd

'/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End_to_End_Chest_Cancer_Detection_ML_Project_using_DVC_and_MLflow/research'

In [3]:
os.chdir("../")

In [4]:
## Entity Creation
## Entity is the return type or the configuration that is to be returned from the config.yaml

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path


##### we read our config.yaml and params.yaml file in constant (init.py)

In [5]:
# Importing both files
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        # Reading YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Creating Artifacts Folder
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion # Imp

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir ,   # Data Ingestion Folder
                source_URL = config.source_URL,   # URL of the dataset
                local_data_file = config.local_data_file, # the file storage location locally
                unzip_dir = config.unzip_dir # unzip data location
            )
            return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [8]:
## Update the Components
#  updating the data ingestion components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    # Download Data
    def download_file(self) -> str:
        """
            Fetch the data from the url 
            """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file # zip file download
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        

        # Unzip the data
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the  data directory
        Function returns None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
# Updating the Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-25 19:30:27,794: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-25 19:30:27,795: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-25 19:30:27,796: INFO: common: created directory at: artifacts]
[2025-09-25 19:30:27,796: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-25 19:30:27,797: INFO: 1686596915: Downloading data from https://drive.google.com/file/d/1eJyF0daLaHgnWmT0ydhNDqykm97saYWi/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1eJyF0daLaHgnWmT0ydhNDqykm97saYWi
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1eJyF0daLaHgnWmT0ydhNDqykm97saYWi&confirm=t&uuid=60fb463c-ae0b-42e6-8058-81ed720db29d
To: /home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End_to_End_Chest_Cancer_Detection_ML_Project_using_DVC_and_MLflow/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [05:09<00:00, 402kB/s]  

[2025-09-25 19:35:40,251: INFO: 1686596915: Download data from https://drive.google.com/file/d/1eJyF0daLaHgnWmT0ydhNDqykm97saYWi/view?usp=sharing into file artifacts/data_ingestion/data.zip]


### Prepare Base Model

We will apply transfer learning where we will make changes in the last(Dense) layer of VGG - 16 

#### All libraries are imported above ,no  need to re import again 

In [10]:
%pwd

'/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End_to_End_Chest_Cancer_Detection_ML_Project_using_DVC_and_MLflow'

In [11]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_model_path : Path
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes : int

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        # Reading YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Creating Artifacts Folder
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
            config = self.config.prepare_base_model # Imp

            create_directories([config.root_dir])

            prepare_base_model_config = PrepareBaseModelConfig(
                root_dir = Path(config.root_dir),
                base_model_path = Path(config.base_model_path),
                updated_model_path = Path(config.updated_base_model_path),
                params_image_size = self.params.IMAGE_SIZE,
                params_learning_rate =  self.params.LEARNING_RATE,
                params_include_top =  self.params.INCLUDE_TOP,
                params_weights =  self.params.WEIGHTS,
                params_classes =  self.params.CLASSES,
            )
            return prepare_base_model_config

In [13]:
import tensorflow as tf

2025-09-25 19:35:40.985614: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-25 19:35:41.010696: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top
        )

        self.model.save(self.config.base_model_path)
    
    
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all: # freeze all layers if true
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0): # freeze till freeze only till the layer mentioned in the argument
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = PrepareBaseModel._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            freeze_all = True,
            freeze_till = None,
            learning_rate = self.config.params_learning_rate
        )

        PrepareBaseModel.save_model(path=self.config.updated_model_path, model = self.full_model)
    

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

In [15]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-09-25 19:35:41,929: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-25 19:35:41,931: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-25 19:35:41,932: INFO: common: created directory at: artifacts]
[2025-09-25 19:35:41,932: INFO: common: created directory at: artifacts/prepare_base_model]


2025-09-25 19:35:41.962339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-25 19:35:41.983492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-25 19:35:41.986207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[2025-09-25 19:35:42,334: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                          

/home/aryan-dhanuka/miniconda3/envs/tf_gpu/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Model Trainer

In [16]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_model_path : Path
    training_data : Path
    validation_data: Path  
    params_epochs : int
    params_batch_size : int
    params_is_augmentation : bool
    params_image_size : list

In [17]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        # Reading YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Creating Artifacts Folder
        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data/train")
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data/test")  # Fixed validation path

        create_directories([Path(training.root_dir)])

        
        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            validation_data = Path(validation_data), 
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        return training_config

In [19]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None 

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_model_path, compile=False)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Reset optimizer
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation data generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        # self.valid_generator = valid_datagenerator.flow_from_directory(
        #     directory=self.config.validation_data,
        #     target_size=self.config.params_image_size[:-1],
        #     batch_size=self.config.params_batch_size,
        #     class_mode="categorical",  # Fixed: Ensures multi-class classification
        #     shuffle=False
        # )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            **dataflow_kwargs,  # UPDATED: Used **dataflow_kwargs to simplify
            class_mode="categorical",  
            shuffle=False
        )

        # Training data generator
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
                
            )
        else:
            train_datagenerator = valid_datagenerator

        # self.train_generator = train_datagenerator.flow_from_directory(
        #     directory=self.config.training_data,
        #     target_size=self.config.params_image_size[:-1],
        #     batch_size=self.config.params_batch_size,
        #     class_mode="categorical",  # Fixed: Ensures correct class label handling
        #     shuffle=True
        # )

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            **dataflow_kwargs,  # UPDATED: Used **dataflow_kwargs
            class_mode="categorical",
            shuffle=True
        )
        print("Train Class Indices:", self.train_generator.class_indices)
        print("Train Num Classes:", self.train_generator.num_classes)
        print("Valid Class Indices:", self.valid_generator.class_indices)
        print("Valid Num Classes:", self.valid_generator.num_classes)

    
    def save_model(self,path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        if not hasattr(self.model, "optimizer"):
            self.model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Reset optimizer
                loss='categorical_crossentropy',
                metrics=['accuracy']
            )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
        


In [20]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2025-09-25 19:35:42,583: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-25 19:35:42,584: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-25 19:35:42,585: INFO: common: created directory at: artifacts]
[2025-09-25 19:35:42,585: INFO: common: created directory at: artifacts/training]
Found 315 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
Train Class Indices: {'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0, 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1, 'normal': 2, 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}
Train Num Classes: 4
Valid Class Indices: {'adenocarcinoma': 0, 'large.cell.carcinoma': 1, 'normal': 2, 'squamous.cell.carcinoma': 3}
Valid Num Classes: 4
Epoch 1/10


2025-09-25 19:35:43.318039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907


 1/38 [..............................] - ETA: 1:21 - loss: 1.6983 - accuracy: 0.1875

2025-09-25 19:35:44.967935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-09-25 19:35:44.978567: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7da33003ff80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-25 19:35:44.978592: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-09-25 19:35:44.981071: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-25 19:35:45.052391: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


38/38 [==============================] - 7s 141ms/step - loss: 1.4286 - accuracy: 0.4791 - val_loss: 1.1832 - val_accuracy: 0.4704
Epoch 2/10
38/38 [==============================] - 5s 131ms/step - loss: 0.9797 - accuracy: 0.5729 - val_loss: 1.1626 - val_accuracy: 0.6086
Epoch 3/10
38/38 [==============================] - 5s 131ms/step - loss: 0.9635 - accuracy: 0.6131 - val_loss: 0.7790 - val_accuracy: 0.6151
Epoch 4/10
38/38 [==============================] - 5s 126ms/step - loss: 0.7163 - accuracy: 0.6817 - val_loss: 0.8165 - val_accuracy: 0.6118
Epoch 5/10
38/38 [==============================] - 5s 128ms/step - loss: 0.6819 - accuracy: 0.7069 - val_loss: 0.8156 - val_accuracy: 0.6217
Epoch 6/10
38/38 [==============================] - 5s 128ms/step - loss: 0.6634 - accuracy: 0.7437 - val_loss: 0.7064 - val_accuracy: 0.6809
Epoch 7/10
38/38 [==============================] - 5s 128ms/step - loss: 0.6142 - accuracy: 0.7404 - val_loss: 0.7124 - val_accuracy: 0.6645
Epoch 8/10
38/38 

### Model Evaluation

In [21]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env file

MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")


In [22]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [ ]:
@dataclass(frozen=True)
class EvaluationConfig:
    def __init__(self, path_of_model, mlflow_url, all_params, params_image_size, params_batch_size, training_data):
        self.path_of_model = path_of_model
        self.mlflow_url = mlflow_url
        self.all_params = all_params
        self.params_image_size = params_image_size
        self.params_batch_size = params_batch_size
        self.training_data = training_data 


In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        # Reading YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Creating Artifacts Folder
        create_directories([self.config.artifacts_root])
    
    def get_evaluated_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data/train",  # now valid
            mlflow_url=MLFLOW_TRACKING_URI,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [25]:
import os
import json
from urllib.parse import urlparse
from pathlib import Path
import mlflow
import tensorflow as tf

class Evaluation:
    def __init__(self, config):
        """
        config: an object with attributes
            - path_of_model: Path to saved Keras model
            - training_data: Path to training/validation dataset
            - all_params: dict of hyperparameters
            - params_image_size: image size tuple
            - params_batch_size: batch size
            - mlflow_url: MLflow tracking URI
        """
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None
        
        mlflow.set_tracking_uri(config.mlflow_url)

    def _valid_generator(self):
        """Create a validation data generator."""
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="categorical"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        print("Valid Class Indices:", self.valid_generator.class_indices)
        print("Number of classes:", len(self.valid_generator.class_indices))

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(path)
        model.summary()
        return model

    def evaluate(self):
        """Evaluate the model on validation data."""
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()

        num_classes = len(self.valid_generator.class_indices)
        if num_classes != 4:
            raise ValueError(f"Expected 4 classes, found {num_classes}. Check your dataset!")

        self.score = self.model.evaluate(self.valid_generator)
        print(f"Loss: {self.score[0]}, Accuracy: {self.score[1]}")
        self._save_score()

    def _save_score(self):
        """Save evaluation metrics to JSON."""
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        with open(Path("scores.json"), "w") as f:
            json.dump(scores, f, indent=4)

    def log_into_mlflow(self):
        """Log metrics and model to MLflow."""
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            # Log hyperparameters
            mlflow.log_params(self.config.all_params)
            # Log metrics
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            # Log model
            mlflow.keras.log_model(self.model, "model") if tracking_url_type_store != "file" else mlflow.keras.log_model(self.model, "model")




In [ ]:

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluated_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluate()
    evaluation._save_score()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e


[2025-09-25 19:36:35,545: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-25 19:36:35,547: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-25 19:36:35,547: INFO: common: created directory at: artifacts]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
           

2025/09/25 19:36:39 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-09-25 19:36:40,531: INFO: builder_impl: Assets written to: /tmp/tmpkxlpoyz9/model/data/model/assets]


2025/09/25 19:36:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run caring-chimp-800 at: https://dagshub.com/AryanDhanuka10/End_to_End_Chest_Cancer_Detection_ML_Project_using_DVC_and_MLflow.mlflow/#/experiments/0/runs/a8eca22590fc439293f6e9d5d3af3d81
🧪 View experiment at: https://dagshub.com/AryanDhanuka10/End_to_End_Chest_Cancer_Detection_ML_Project_using_DVC_and_MLflow.mlflow/#/experiments/0
